# Persona Generation Workflow - Research Replication Guide

This notebook demonstrates how to replicate the persona generation process from **"The Personality Trap: Evaluating Psychological Profiles in Large Language Models"** research.

## Overview

This workflow guides you through:

1. **Schema Setup**: Create a new experimental PostgreSQL schema for your data
2. **Reference Data**: Select and copy reference questionnaire responses 
3. **Persona Generation**: Generate AI personas using different LLM models
4. **Verification**: Inspect and validate the generated personas

## Prerequisites

Before running this notebook:

- ✅ PostgreSQL database is running (via Docker or external service)
- ✅ Configuration file exists (`.yaml` or `.yaml.example`)
- ✅ LLM API keys are configured (OpenAI, AWS Bedrock, etc.)
- ✅ Python dependencies installed via `uv sync` or `pip install .`

## Database Schema Architecture

The system uses two schemas:

- **Production schema** (`personality_trap`): Contains the original research data - READ ONLY
- **Experimental schema** (configured via `schema.target_schema`): Your working area for replication

This notebook will create and populate the experimental schema without affecting production data.

---

## Step 1: Configure the Backend and Database Connection

This cell imports required libraries and sets up the database connection. We configure logging to track the persona generation process and establish connections to both the research schema (read-only) and your experimental schema (read-write).

**Key components:**
- `ConfigManager`: Loads database credentials, API keys, and schema configuration from YAML
- `DatabaseHandler`: Manages PostgreSQL connections and schema operations
- `Population`: Handles demographic data and persona storage
- `PersonaGenerator`: Orchestrates the LLM-based persona generation process

**Schema configuration:**
- All schema settings are read from `.yaml` file (no environment variables)
- `schema.default_schema`: Production/research data (read-only)
- `schema.target_schema`: Your experimental working schema (read-write)

In [1]:
import logging
from pathlib import Path
from typing import List

import pandas as pd
from IPython.display import display
from sqlalchemy import text

# Import configuration and core modules
from personas_backend.utils.config import ConfigManager
from personas_backend.core.enums import ModelID, PersonaGenerationCondition
from personas_backend.db.db_handler import DatabaseHandler
from personas_backend.persona_generator import PersonaGenerator
from personas_backend.population.population import Population

# Configure logging to track progress
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Load configuration from YAML (all schema settings read from here)
config = ConfigManager()

# Initialize database handler
db_handler = DatabaseHandler(config_manager=config)
population_handler = Population(db_handler=db_handler)

# Schema configuration (read from YAML)
RESEARCH_SCHEMA = config.schema_config.default_schema  # Original research data (READ-ONLY)
EXPERIMENTAL_SCHEMA = config.schema_config.target_schema  # Your working schema (READ-WRITE)

# Reference population to replicate (from research data)
REFERENCE_POPULATION = 'spain826'
OUTPUT_TABLE = 'personas'

print(f"✅ Configuration loaded successfully from YAML")
print(f"📊 Research schema (read-only): {RESEARCH_SCHEMA}")
print(f"🧪 Experimental schema (working): {EXPERIMENTAL_SCHEMA}")
print(f"🌍 Reference population: {REFERENCE_POPULATION}")
print(f"📝 Output table: {OUTPUT_TABLE}")

2025-10-09 18:21:16,114 - personas_backend.db.db_handler - INFO - Connected to the database!


✅ Configuration loaded successfully from YAML
📊 Research schema (read-only): personality_trap
🧪 Experimental schema (working): my_schema_v00
🌍 Reference population: spain826
📝 Output table: personas


## Step 2: Create and Initialize Experimental Schema

Before generating personas, we need to create a dedicated schema for your experimental data. This ensures:

- **Isolation**: Your work won't affect the original research data
- **Reproducibility**: Clean slate for replicating the research process  
- **Schema consistency**: All required tables are created via Alembic migrations

**What happens here:**
1. Check if the experimental schema already exists
2. Run Alembic migrations to create the schema and all necessary tables
3. Verify the schema and tables were created successfully

**Tables created:**
- `personas`: Stores generated AI personas with demographics
- `reference_questionnaires`: Personality questionnaire responses
- `experiments_list`: Individual experiment runs
- `experiments_groups`: Batches of related experiments
- `eval_questionnaires`: Questionnaire answers from LLMs
- `experiment_request_metadata`: LLM API request/response logs

**How Alembic handles multiple schemas:**
- Each schema gets its own `alembic_version` table for independent version tracking
- The `version_table_schema` parameter ensures migrations are schema-specific
- This allows the same migration to be applied to different schemas independently

In [2]:
# Check if experimental schema exists
schema_exists = db_handler.schema_exists(EXPERIMENTAL_SCHEMA)

if schema_exists:
    print(f"ℹ️  Schema '{EXPERIMENTAL_SCHEMA}' already exists")
    print(f"   Applying any pending migrations...")
else:
    print(f"🆕 Creating new schema '{EXPERIMENTAL_SCHEMA}'...")

# Run Alembic migrations to create/update schema
# Alembic will:
# 1. Create the schema if it doesn't exist (via env.py)
# 2. Create schema-specific alembic_version table
# 3. Apply all pending migrations to create tables
try:
    db_handler.run_migrations(schema=EXPERIMENTAL_SCHEMA)
    print(f"✅ Migrations applied successfully to '{EXPERIMENTAL_SCHEMA}'")
except Exception as e:
    print(f"❌ Migration failed: {e}")
    raise

# Verify schema was created
if not db_handler.schema_exists(EXPERIMENTAL_SCHEMA):
    raise RuntimeError(f"Schema '{EXPERIMENTAL_SCHEMA}' was not created. Check database permissions.")

# Verify critical tables exist
critical_tables = ['personas', 'reference_questionnaires', 'experiments_list', 
                   'experiments_groups', 'eval_questionnaires']
missing_tables = []

for table_name in critical_tables:
    if not db_handler.check_table_exists(EXPERIMENTAL_SCHEMA, table_name):
        missing_tables.append(table_name)

if missing_tables:
    print(f"⚠️  Warning: Missing tables: {', '.join(missing_tables)}")
else:
    print(f"✅ All critical tables verified in '{EXPERIMENTAL_SCHEMA}'")

# Display schema summary
with db_handler.connection.connect() as conn:
    tables_query = text(f"""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = :schema
        ORDER BY table_name
    """)
    tables_df = pd.read_sql_query(tables_query, conn, params={"schema": EXPERIMENTAL_SCHEMA})
    
print(f"\n📋 Tables in '{EXPERIMENTAL_SCHEMA}' schema:")
display(tables_df)

🆕 Creating new schema 'my_schema_v00'...


INFO  [alembic.runtime.migration] Context impl PostgresqlImpl.
INFO  [alembic.runtime.migration] Context impl PostgresqlImpl.
INFO  [alembic.runtime.migration] Will assume transactional DDL.
INFO  [alembic.runtime.migration] Will assume transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 001_initial_schema, Initial schema with auto-increment and server-side timestamps
INFO  [alembic.runtime.migration] Running upgrade  -> 001_initial_schema, Initial schema with auto-increment and server-side timestamps


✅ Migrations applied successfully to 'my_schema_v00'
✅ All critical tables verified in 'my_schema_v00'

📋 Tables in 'my_schema_v00' schema:


,table_name
0,alembic_version
1,eval_questionnaires
2,experiment_request_metadata
3,experiments_groups
4,experiments_list
5,personas
6,questionnaire
7,random_questionnaires
8,reference_questionnaires


## Step 3: Select and Copy Reference Questionnaire Data

To generate personas, we need reference personality questionnaire responses. This step:

1. **Selects** a subset of personality IDs from the original research data
2. **Copies** those questionnaire responses to your experimental schema
3. **Prevents duplicates** by checking existing data before inserting

**Configuration:**
- `N_PERSONALITIES`: Number of reference personalities to use (adjust as needed)
- Larger samples → more diverse personas, but longer processing time
- Start with 2-5 for testing, scale up for full replication

**Data flow:**
```
research_schema.reference_questionnaires (READ)
    ↓ SELECT distinct personality_id
    ↓ COPY questionnaire rows
experimental_schema.reference_questionnaires (WRITE)
```

In [3]:
# Configure sample size
N_PERSONALITIES = 2  # Start small for testing; increase for full replication

print(f"🎯 Selecting {N_PERSONALITIES} reference personalities from research data...")

# Step 1: Select personality IDs from research schema
pick_ids_sql = f"""
SELECT DISTINCT personality_id
FROM {RESEARCH_SCHEMA}.reference_questionnaires
ORDER BY personality_id
LIMIT {int(N_PERSONALITIES)}
"""

with db_handler.connection.connect() as conn:
    src_ids_df = pd.read_sql_query(pick_ids_sql, conn)

if src_ids_df.empty:
    raise ValueError(
        f"No personality_id found in {RESEARCH_SCHEMA}.reference_questionnaires. "
        f"Check RESEARCH_SCHEMA or data availability."
    )

selected_ids = src_ids_df["personality_id"].astype(int).tolist()
print(f"✅ Selected personality IDs: {selected_ids}")

# Step 2: Verify target table exists
if not db_handler.check_table_exists(EXPERIMENTAL_SCHEMA, "reference_questionnaires"):
    raise RuntimeError(
        f"Table {EXPERIMENTAL_SCHEMA}.reference_questionnaires not found. "
        f"Run migrations first (previous cell)."
    )

# Step 3: Copy questionnaire data to experimental schema
ids_csv = ", ".join(str(pid) for pid in selected_ids)
insert_rq_sql = f"""
INSERT INTO {EXPERIMENTAL_SCHEMA}.reference_questionnaires (
    personality_id,
    question_number,
    question,
    category,
    key,
    answer
)
SELECT
    rq.personality_id,
    rq.question_number,
    rq.question,
    rq.category,
    rq.key,
    rq.answer
FROM {RESEARCH_SCHEMA}.reference_questionnaires rq
WHERE rq.personality_id IN ({ids_csv})
  AND NOT EXISTS (
      SELECT 1
      FROM {EXPERIMENTAL_SCHEMA}.reference_questionnaires t
      WHERE t.personality_id = rq.personality_id
        AND t.question_number = rq.question_number
  )
"""

with db_handler.connection.connect() as conn:
    result = conn.execute(text(insert_rq_sql))
    try:
        inserted_rq = result.rowcount if result.rowcount is not None else 0
    except Exception:
        inserted_rq = 0
    conn.commit()

print(f"✅ Inserted {inserted_rq} questionnaire rows into {EXPERIMENTAL_SCHEMA}.reference_questionnaires")

# Step 4: Load questionnaire data for persona generation
# IMPORTANT: persona_id and personality_id both reference the same unique identifier
# - Both columns are required for internal generator compatibility
# - persona_id: used for iteration over unique personalities
# - personality_id: used for filtering questionnaire responses
# - In production, these would come from different tables; here we alias for simplicity
questionnaire_sql = f"""
SELECT 
    personality_id,
    personality_id AS persona_id,
    question_number, 
    question, 
    category, 
    key, 
    answer
FROM {EXPERIMENTAL_SCHEMA}.reference_questionnaires
WHERE personality_id IN ({ids_csv})
ORDER BY personality_id, question_number
"""

questionnaire_df = pd.read_sql_query(questionnaire_sql, db_handler.connection)
questionnaire_df["experiment_id"] = 0  # Synthetic ID (not used in persona generation)

# Defensive: some legacy paths expect a 'description' column; provide empty fallback
if "description" not in questionnaire_df.columns:
    questionnaire_df["description"] = ""

print(f"\n📊 Loaded {len(questionnaire_df)} questionnaire responses:")
print(f"   • {questionnaire_df['persona_id'].nunique()} unique personalities")
print(f"   • {questionnaire_df['question_number'].nunique()} questions per personality")
print(f"   • Columns: {list(questionnaire_df.columns)}")

# Preview sample
display(questionnaire_df.head(10))

# Store for next steps
target_personalities = selected_ids


🎯 Selecting 2 reference personalities from research data...
✅ Selected personality IDs: [1, 2]
✅ Inserted 48 questionnaire rows into my_schema_v00.reference_questionnaires

📊 Loaded 48 questionnaire responses:
   • 2 unique personalities
   • 24 questions per personality
   • Columns: ['personality_id', 'persona_id', 'question_number', 'question', 'category', 'key', 'answer', 'experiment_id', 'description']


,personality_id,persona_id,question_number,question,category,key,answer,experiment_id,description
0,1,1,1,Does your mood often go up and down?,N,True,True,0,
1,1,1,2,Are you a talkative person?,E,True,False,0,
2,1,1,3,Would being in debt worry you?,P,False,True,0,
3,1,1,4,Are you rather lively?,E,True,False,0,
4,1,1,5,Were you ever greedy by helping yourself to mo...,L,False,False,0,
5,1,1,6,Would you take drugs which may have strange or...,P,True,False,0,
6,1,1,7,Have you ever blamed someone for doing somethi...,L,False,False,0,
7,1,1,8,Do you prefer to go your own way rather than a...,P,True,False,0,
8,1,1,9,Do you often feel 'fed-up'?,N,True,True,0,
9,1,1,10,Have you ever taken anything (even a pin or bu...,L,False,False,0,


## Step 4: Initialize Persona Generator

Now we'll create the `PersonaGenerator` instance that orchestrates LLM-based persona creation. This component:

- **Reads** reference questionnaire responses (from previous step)
- **Generates** persona descriptions using LLM APIs (GPT, Claude, Llama)
- **Persists** generated personas to the database with proper schema isolation
- **Tracks** generation progress and handles API failures gracefully

The generator is pre-configured with your experimental schema, ensuring all generated personas are stored separately from research data.

In [4]:
# Verify schema configuration from YAML
print(f"🔍 Current schema configuration:")
print(f"   • default_schema (from YAML): {config.schema_config.default_schema}")
print(f"   • target_schema (from YAML): {config.schema_config.target_schema}")
print(f"   • EXPERIMENTAL_SCHEMA (active): {EXPERIMENTAL_SCHEMA}")

# Initialize the persona generator (will use EXPERIMENTAL_SCHEMA)
generator = PersonaGenerator(
    db_handler=db_handler,
    population_handler=population_handler,
    config_manager=config,
    ref_population=REFERENCE_POPULATION,
    schema=EXPERIMENTAL_SCHEMA,
)

print(f"\n✅ PersonaGenerator initialized")
print(f"   • Target schema: {EXPERIMENTAL_SCHEMA}")
print(f"   • Reference population: {REFERENCE_POPULATION}")
print(f"   • Output table: {OUTPUT_TABLE}")

🔍 Current schema configuration:
   • default_schema (from YAML): personality_trap
   • target_schema (from YAML): my_schema_v00
   • EXPERIMENTAL_SCHEMA (active): my_schema_v00

✅ PersonaGenerator initialized
   • Target schema: my_schema_v00
   • Reference population: spain826
   • Output table: personas


## Step 5: Generate Baseline Personas

**Baseline personas** are generated by prompting LLMs with reference questionnaire responses to create realistic human personas.

**Process:**
1. For each reference personality + model combination:
   - LLM receives questionnaire answers (Big Five personality traits)
   - LLM generates a persona description (name, age, demographics, background)
   - System stores persona in `{experimental_schema}.personas` table

**Model Selection:**
- Start with 1-2 models for testing
- Expand to all models for full replication
- Each model may generate different personas from the same questionnaire

**Performance notes:**
- `max_workers=2`: Process 2 personas concurrently (adjust based on API rate limits)
- `repetitions=1`: Generate 1 persona per personality-model pair
- Progress is logged in real-time; check for API errors

**Important:** The `questionnaire_df` must have **both** `persona_id` and `personality_id` columns:
- `persona_id`: Used by the persona lookup logic
- `personality_id`: Used by the questionnaire filtering logic
- Both point to the same values (we create `persona_id` as an alias)

**Warning:** This may take several minutes depending on the number of personalities and models.


In [5]:
# Select models for baseline persona generation
# Start with 1 for testing; add more for full replication
baseline_models = [
    # ModelID.GPT4O,  # OpenAI GPT-4o (recommended for first run)
    ModelID.GPT35,          # OpenAI GPT-3.5 Turbo
    # ModelID.CLAUDE35_SONNET, # Anthropic Claude 3.5 Sonnet (requires AWS Bedrock)
    # ModelID.LLAMA3_23B,     # Meta Llama 3.2 3B (requires Bedrock or local hosting)
    # ModelID.LLAMA3_170B,    # Meta Llama 3.1 70B (requires Bedrock)
]

print(f"🎯 Generating baseline personas for {len(baseline_models)} model(s):")
for model in baseline_models:
    print(f"   • {model.value}")

print(f"\n⏳ Starting persona generation (this may take several minutes)...")
print(f"   • Personalities to process: {len(target_personalities)}")
print(f"   • Total personas to generate: {len(target_personalities) * len(baseline_models)}")

# Generate personas using the package API
try:
    generator.generate_personas(
        models=baseline_models,
        population_df=questionnaire_df,
        new_population_table=OUTPUT_TABLE,
        exp_df=questionnaire_df,
        max_workers=2,           # Concurrent workers (adjust based on API rate limits)
        repetitions=1,           # Personas per personality-model pair
        allow_experimental=True  # Enable experimental schema usage
    )
    print(f"\n✅ Baseline persona generation completed successfully!")
    
except Exception as e:
    print(f"\n❌ Persona generation failed: {e}")
    print(f"   Check logs for details. Common issues:")
    print(f"   • Missing API keys in configuration")
    print(f"   • API rate limits exceeded")
    print(f"   • Network connectivity issues")
    raise

🎯 Generating baseline personas for 1 model(s):
   • gpt35

⏳ Starting persona generation (this may take several minutes)...
   • Personalities to process: 2
   • Total personas to generate: 2

✅ Baseline persona generation completed successfully!


## Step 6: Generate Borderline Personas (Optional)

**Borderline personas** are experimental variants where personality traits are artificially adjusted to extreme values. This is used in the research to study how personality extremes affect LLM behavior.

**Conditions:**
- `MAX_N` (Maximally Neurotic): Neuroticism trait maximized, others minimized
- `MAX_P` (Maximally Psychotic): Psychoticism trait maximized, others minimized  
- Additional conditions can be defined based on research needs

**Use case:** This step is optional and primarily for replicating the full research experiments. Skip this if you only need baseline personas.

**Process:**
1. System modifies questionnaire answers to create extreme personality profiles
2. LLM generates personas based on the modified questionnaires
3. Personas are tagged with the borderline condition in the database

**Note:** Borderline generation is typically done with a single model (GPT-4o recommended) to maintain consistency.

In [6]:
# OPTIONAL: Generate borderline personas for experimental research
# Set GENERATE_BORDERLINE = True to enable this step
GENERATE_BORDERLINE = True  # Change to True if needed

if GENERATE_BORDERLINE:
    # Select model and conditions
    borderline_model = ModelID.GPT4O
    borderline_conditions = [
        PersonaGenerationCondition.MAX_N,  # Maximally Neurotic
        PersonaGenerationCondition.MAX_P,  # Maximally Psychotic
    ]
    
    print(f"🎯 Generating borderline personas:")
    print(f"   • Model: {borderline_model.value}")
    print(f"   • Conditions: {[c.value for c in borderline_conditions]}")
    print(f"   • Personalities: {len(target_personalities)}")
    print(f"\n⏳ Starting borderline generation...")
    
    try:
        generator.generate_borderline_personas(
            model=borderline_model,
            borderline_list=borderline_conditions,
            personas_list=[str(pid) for pid in selected_ids],
            population_df=questionnaire_df,
            new_population_table=OUTPUT_TABLE,
            exp_df=questionnaire_df,
            max_workers=2,
            repetitions=1,
            allow_experimental=True
        )
        print(f"✅ Borderline persona generation completed!")
        
    except Exception as e:
        print(f"❌ Borderline generation failed: {e}")
        raise
else:
    print(f"ℹ️  Borderline persona generation skipped (GENERATE_BORDERLINE=False)")
    print(f"   Set GENERATE_BORDERLINE=True to enable experimental persona variants")

🎯 Generating borderline personas:
   • Model: gpt4o
   • Conditions: ['max_N', 'max_P']
   • Personalities: 2

⏳ Starting borderline generation...
✅ Borderline persona generation completed!


## Step 7: Verify Generated Personas

Query the experimental schema to confirm personas were generated and stored correctly. This validation step:

1. **Counts** total personas generated by model and population
2. **Previews** sample persona records with demographics
3. **Validates** data integrity (no missing critical fields)

**What to look for:**
- Expected number of personas (personalities × models × repetitions)
- Populated demographic fields (name, age, gender, race, etc.)
- Correct `population` tags (e.g., `generated_gpt4o_spain826`)
- Valid `ref_personality_id` linking back to reference data

**Troubleshooting:**
- If count is lower than expected, check logs for API failures
- Missing demographics may indicate LLM parsing errors
- Empty results suggest schema configuration issues

In [7]:
# Additional diagnostics: Check what's in the reference questionnaires
print(f"🔍 Debugging: Checking data availability...")

# Check reference questionnaires
ref_check_sql = f"""
SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT personality_id) as unique_personalities,
    MIN(personality_id) as min_id,
    MAX(personality_id) as max_id
FROM {EXPERIMENTAL_SCHEMA}.reference_questionnaires
"""

ref_stats = pd.read_sql_query(ref_check_sql, db_handler.connection)

print(f"\n📊 Reference questionnaires in {EXPERIMENTAL_SCHEMA}:")
display(ref_stats)

if ref_stats.iloc[0]['total_rows'] == 0:
    print(f"\n❌ No reference questionnaire data found!")
    print(f"   Go back and run Step 3 to copy reference data")
else:
    print(f"\n✅ Reference data available")
    print(f"   Ready for persona generation")
    
# Check if questionnaire_df still has the right columns
print(f"\n🔍 Checking questionnaire_df DataFrame:")
print(f"   Columns: {list(questionnaire_df.columns)}")
print(f"   Has 'persona_id': {'persona_id' in questionnaire_df.columns}")
print(f"   Has 'personality_id': {'personality_id' in questionnaire_df.columns}")
print(f"   Shape: {questionnaire_df.shape}")

# This is the CRITICAL check - both columns must exist for generation to work!
if 'persona_id' in questionnaire_df.columns and 'personality_id' in questionnaire_df.columns:
    print(f"\n✅ Both required columns present!")
    print(f"   PersonaGenerator should work correctly")
    print(f"\n📋 Sample data:")
    display(questionnaire_df[['personality_id', 'persona_id', 'question_number']].head(5))
else:
    print(f"\n❌ MISSING REQUIRED COLUMNS!")
    if 'persona_id' not in questionnaire_df.columns:
        print(f"   Missing 'persona_id' - needed for population_df iteration")
    if 'personality_id' not in questionnaire_df.columns:
        print(f"   Missing 'personality_id' - needed for exp_df filtering")
    print(f"\n🔧 FIX: Re-run Step 3 to reload the data with both columns")


🔍 Debugging: Checking data availability...

📊 Reference questionnaires in my_schema_v00:


,total_rows,unique_personalities,min_id,max_id
0,48,2,1,2



✅ Reference data available
   Ready for persona generation

🔍 Checking questionnaire_df DataFrame:
   Columns: ['personality_id', 'persona_id', 'question_number', 'question', 'category', 'key', 'answer', 'experiment_id', 'description']
   Has 'persona_id': True
   Has 'personality_id': True
   Shape: (48, 9)

✅ Both required columns present!
   PersonaGenerator should work correctly

📋 Sample data:


,personality_id,persona_id,question_number
0,1,1,1
1,1,1,2
2,1,1,3
3,1,1,4
4,1,1,5


## Step 8: Next Steps

🎉 **Persona generation complete!** You now have AI-generated personas stored in your experimental schema.

### What's next?

1. **Questionnaire Experiments** (see `questionnaires_experiments.ipynb`):
   - Register experiment groups for your generated personas
   - Administer personality questionnaires to LLMs impersonating the personas
   - Analyze how LLM responses compare to the original reference personalities

2. **Data Analysis**:
   - Compare demographic distributions across models
   - Analyze personality trait consistency
   - Study bias patterns in persona generation

3. **Export Data**:
   ```python
   # Export personas to CSV for external analysis
   results_df.to_csv('generated_personas.csv', index=False)
   ```

### Cleanup (Optional)

To close the database connection cleanly:

In [8]:
db_handler.close_connection()
print("✅ Database connection closed")
print("\n👉 Next: Open 'questionnaires_experiments.ipynb' to run questionnaire evaluations")

✅ Database connection closed

👉 Next: Open 'questionnaires_experiments.ipynb' to run questionnaire evaluations
